<a href="https://colab.research.google.com/github/ggeorg0/rag-finetuned-mistral7b/blob/main/model_fine-tuning-w-Qlora.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Sometimes Google Colab throws encding error. This statement used to it.
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

## Установка и импорт зависимостей

In [ ]:
!pip install auto-gptq
!pip install optimum
!pip install bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.5/23.5 MB 55.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 28.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.2/13.2 MB 85.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.1/199.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 65.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 63.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 85.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 14.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━

In [ ]:
# resolving "No inf checks were recorded for this optimizer." issue
!pip uninstall torch -y
!pip install torch==2.1

Found existing installation: torch 2.2.1+cu121
Uninstalling torch-2.2.1+cu121:
  Successfully uninstalled torch-2.2.1+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 670.2/670.2 MB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 MB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 MB 18.8 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 2.2.0
    Uninstalling triton-2.2.0:
      Successfully uninstalled triton-2.2.0
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.19.3
    Uninstalling nvidia-nccl-cu12-2.19.3:
      Successfully uninstalled nvidia-nccl-cu12-2.19.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.1+cu121 requires torch==2.2.1, but you have torch 2.1.0 which is incompatible.
torchtext 0.17.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from peft import prepare_model_for_kbit_training
from peft import LoraConfig, get_peft_model
from datasets import load_dataset
import transformers

### Load model

In [ ]:
model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto", # automatically figures out how to best use CPU + GPU for loading model
                                             trust_remote_code=False, # prevents running custom model files on your machine
                                             revision="gptq-4bit-32g-actorder_True") # which version of model to use in repo

/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:4193: FutureWarning: `_is_quantized_training_enabled` is going to be deprecated in transformers 4.39.0. Please use `model.hf_quantizer.is_trainable` instead
  warnings.warn(


In [ ]:
# test query
user_query = "Что делать, если потерял паспорт? Как сделать новый?"

### Загрузка токенайзера модели

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)
tokenizer.pad_token = tokenizer.eos_token

### Проверка базовой модели

In [ ]:
model.eval() # model in evaluation mode (dropout modules are deactivated)

Небольшой промт-инженеринг. Здесь задается шаблон, к которому будет прикрепляться сообщение пользователя. Сам пользователь не будет видеть этот текст

In [ ]:
intstructions_string = "Вы - русскоязычный ИИ ассистент, который помогает пользователям находить файлы, \
отвечать на их вопросы и поддерживать диалог. \
Вы имеете доступ к базе данных, которая автоматически покажет результаты поиска, \
если найдется какая-либо информация. Помните, что не все результаты поиска могут быть релевантны. \
Адаптируйтесь к запросам пользователей и предоставляйте понятные и полезные ответы на вопросы, \
поддерживая диалог на русском языке. \
\n\
\n\
\n\
Сообщение пользователя:"

def prompt_template(user_query):
    return f'''[INST] {intstructions_string} \n{user_query} \n[/INST]'''


prompt = prompt_template(user_query)
print(prompt)

[INST] Вы - русскоязычный ИИ ассистент, который помогает пользователям находить файлы, отвечать на их вопросы и поддерживать диалог. Вы имеете доступ к базе данных, которая автоматически покажет результаты поиска, если найдется какая-либо информация. Помните, что не все результаты поиска могут быть релевантны. Адаптируйтесь к запросам пользователей и предоставляйте понятные и полезные ответы на вопросы, поддерживая диалог на русском языке. 


Сообщение пользователя: 
Что делать, если потерял паспорт? Как сделать новый? 
[/INST]


In [ ]:
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Вы - русскоязычный ИИ ассистент, который помогает пользователям находить файлы, отвечать на их вопросы и поддерживать диалог. Вы имеете доступ к базе данных, которая автоматически покажет результаты поиска, если найдется какая-либо информация. Помните, что не все результаты поиска могут быть релевантны. Адаптируйтесь к запросам пользователей и предоставляйте понятные и полезные ответы на вопросы, поддерживая диалог на русском языке. 


Сообщение пользователя: 
Что делать, если потерял паспорт? Как сделать новый? 
[/INST] При ло /******/тении паспорта необходимо сделать следующие шаги:

1. Сообщить о потере документа в органе власти, который выдавал паспорт (ОВД, ФМС, МВД). Это необходимо для предотвращения возможных случаев фальсификации документа.
2. Подготовить необходимые документы для получения нового паспорта:
   - Для граждан Российской Федерации: свидетельство о рождении, свидетельство о браке (если женат), документы на детей (если есть), документы на жилье, личное пи

**Как мы видим, модель сгенерировала какой-то странный резльутат, хотя и поняла основной смысл. Эту проблему можно решить с помощью файн-тьюнинга (дообучения) модели.**

### Подготовка датасета Yandex Q

In [ ]:
from datasets import load_dataset

dataset = load_dataset("its5Q/yandex-q", split="train")
dataset

Dataset({
    features: ['description', 'question', 'answer'],
    num_rows: 664246
})

Отфильтруем данные по размеру ответа. А также оставим все вопросы, у которых есть описание, для того, чтобы в будущем использовать эту информацию в качестве "поиска по базе данных". Может быть это не совсем идеальный подход, но на мой взгляд он позволяет "приучить" модель к тому, что у неё есть доступ какой-то системе с дополнительной информацией.

In [ ]:
low_threshould = 400
high_threshould = 1600

clean_dataset = dataset.filter(lambda ds: low_threshould <= len(ds['answer']) <= high_threshould
                                          and len(ds['description']) > 100)

In [ ]:
clean_dataset

Dataset({
    features: ['description', 'question', 'answer'],
    num_rows: 43092
})

Скачаем файлы "плохих слов" и популярных имен

In [ ]:
!wget https://raw.githubusercontent.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/5faf2ba42d7b1c0977169ec3611df25a3c08eb13/ru
!wget https://raw.githubusercontent.com/sorokinpf/russian_names/master/russian_names.txt
!mv ru List-of-Bad-Words.txt
!mv russian_names.txt popular_names.txt
!ls

--2024-03-31 09:35:40--  https://raw.githubusercontent.com/LDNOOBW/List-of-Dirty-Naughty-Obscene-and-Otherwise-Bad-Words/5faf2ba42d7b1c0977169ec3611df25a3c08eb13/ru
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2223 (2.2K) [text/plain]
Saving to: ‘ru’

ru                  100%[===================>]   2.17K  --.-KB/s    in 0s      

2024-03-31 09:35:40 (56.9 MB/s) - ‘ru’ saved [2223/2223]

--2024-03-31 09:35:40--  https://raw.githubusercontent.com/sorokinpf/russian_names/master/russian_names.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 2

In [ ]:
def read_file_with_words(filename):
    words = []
    with open(filename, 'r', encoding='UTF-8') as f:
        for line in f:
            words.append(line.strip())
    return words

bad_words = read_file_with_words('List-of-Bad-Words.txt')
popular_names = read_file_with_words('popular_names.txt')

In [ ]:
def no_bad_words(text, restricted_words):
    text = text.upper()
    for rword in restricted_words:
        if rword.upper() in text:
            return False
    return True

# remove bad words
clean_dataset = clean_dataset.filter(lambda ds: no_bad_words(ds['answer'], bad_words))

In [ ]:
clean_dataset

Dataset({
    features: ['description', 'question', 'answer'],
    num_rows: 33192
})

Здесь вы можете видеть слова, которые, по моему мнению, встречаются в ответах низкого качества.

In [ ]:
controversial_words = [
    "общение", "диалог", "разговор", "мотивация",
    "стыд", "неловкость", "одиночество", "изоляция",
    "отсутствие друзей", "социальная изоляция", "сомнения",
    "ненависть к себе", "стресс", "тревожность", "низкое самооценка",
    "отсутствие уверенности", "дети", "воспитание", "поведение",
    "образование", "родители", "проблемы в школе", "забота о ребенке",
    "отношения", "разрыв", "недоверие", "ревность", "изменяет", "одноклассники",
    "партнер", "вторая половинка", "маму", "маме", "семье", "9 лет"
    "10 лет", "11 лет", "12 лет", "13 лет", "14 лет", "15 лет"
]
clean_dataset = clean_dataset.filter(lambda ds: no_bad_words(ds['answer'], controversial_words))
clean_dataset = clean_dataset.filter(lambda ds: no_bad_words(ds['description'], controversial_words))

Filter:   0%|          | 0/33192 [00:00<?, ? examples/s]

Filter:   0%|          | 0/25716 [00:00<?, ? examples/s]

In [ ]:
clean_dataset

Dataset({
    features: ['description', 'question', 'answer'],
    num_rows: 22830
})

In [ ]:
clean_dataset[19]

{'description': 'С 1.01.2020 г. вроде как вступают в силу изменения по вывозу мусора. Соответственно все должны заключить договор с региональным оператором. И вроде как заключать договор лично не нужно так как он публичный и публикуется в местных СМИ. Отсюда вопрос с какого момента договор можно считать заключённым? Можно ли считать исполнение публичного договора если он опубликован в местной газете тиражом 5 тысяч копий, а в районе более 20 тысяч домовладений и более 110 тысяч жителей? Есть ли ответственность за не заключение договора? Откуда оператор будет знать норму на домовладение (раньше оплачивалось по количеству проживающих) и можно ли привлечь оператора или иное лицо предоставившее оператору личные данные для заключение договора (ФИО, количество проживающих и т.д. что может быть отображено в квитанциях)? В общем вопросов больше чем ответов.',
 'question': 'С 1.01.2020 г. вроде как вступают в силу изменения по вывозу мусора. Вроде как все должны заключить договор с региональным

Удалим привественные предложения, которые содержат имена. Пример: "Здравствуйте, Семён!"

In [ ]:
import string

def remove_greetings(sample, names):
    answer = sample['answer']
    starting_words = answer.translate(str.maketrans('', '', string.punctuation)).split()
    if any(n in starting_words for n in names):
        greet_index = 50
        for ch in '!.?,':
            index = answer.find(ch, 0, 50)
            if index != -1 and index < greet_index:
                greet_index = index
        if greet_index == 50:
            return sample
        # greet_index+2 to remove punktuation and whitespace
        sample['answer'] = answer[greet_index+2:]
    return sample

# remove greetings
clean_dataset = clean_dataset.map(lambda x: remove_greetings(x, popular_names), batched=False)

Map:   0%|          | 0/26769 [00:00<?, ? examples/s]

In [ ]:
clean_dataset[19]

{'description': 'С 1.01.2020 г. вроде как вступают в силу изменения по вывозу мусора. Соответственно все должны заключить договор с региональным оператором. И вроде как заключать договор лично не нужно так как он публичный и публикуется в местных СМИ. Отсюда вопрос с какого момента договор можно считать заключённым? Можно ли считать исполнение публичного договора если он опубликован в местной газете тиражом 5 тысяч копий, а в районе более 20 тысяч домовладений и более 110 тысяч жителей? Есть ли ответственность за не заключение договора? Откуда оператор будет знать норму на домовладение (раньше оплачивалось по количеству проживающих) и можно ли привлечь оператора или иное лицо предоставившее оператору личные данные для заключение договора (ФИО, количество проживающих и т.д. что может быть отображено в квитанциях)? В общем вопросов больше чем ответов.',
 'question': 'С 1.01.2020 г. вроде как вступают в силу изменения по вывозу мусора. Вроде как все должны заключить договор с региональным

Шаблон с которым мы будем обучать модель.

In [ ]:
sample_template = """<s>[INST]Вы - русскоязычный ИИ ассистент, который помогает пользователям находить файлы, \
отвечать на их вопросы и поддерживать диалог.


дополнительная информация:
{description}


сообщение пользователя:
{message}[/INST]{answer}
<s>

"""

In [ ]:
def prepare_and_tokenize(sample):
    # extract text
    text = sample_template.format(
        description=sample["description"],
        message=sample["question"],
        answer=sample["answer"]
    )

    #tokenize and truncate text
    tokenizer.truncation_side = "left"
    tokenized_inputs = tokenizer(
        text,
        return_tensors="np",
        truncation=True,
        max_length=1024*2
    )

    return tokenized_inputs

In [ ]:
train, test = clean_dataset.train_test_split(test_size=0.1).values()
print(f"train: {len(train)}, test: {len(test)}")

train: 24092, test: 2677


In [ ]:
train = train.map(prepare_and_tokenize, batched=True, remove_columns=["description", "question", "answer"])
test = test.map(prepare_and_tokenize, batched=True, remove_columns=["description", "question", "answer"])


Map:   0%|          | 0/24092 [00:00<?, ? examples/s]

Map:   0%|          | 0/2677 [00:00<?, ? examples/s]

In [ ]:
# setting pad token
tokenizer.pad_token = tokenizer.eos_token
# data collator
data_collator = transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)


### Prepare Model for Training

In [ ]:
model.train() # model in training mode (dropout modules are activated)

# enable gradient check pointing
model.gradient_checkpointing_enable()

# enable quantized training
model = prepare_model_for_kbit_training(model)

In [ ]:
# LoRA config
config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

# LoRA trainable version of model
model = get_peft_model(model, config)

# trainable parameter count
model.print_trainable_parameters()

trainable params: 2,097,152 || all params: 264,507,392 || trainable%: 0.7928519441906561


### Fine-tuning Model

In [ ]:
# hyperparameters
lr = 5e-5
batch_size = 8
num_epochs = 60

lora_model_name = "qlora-mistral-hackatone-yandexq"

# define training arguments
training_args = transformers.TrainingArguments(
    output_dir=lora_model_name,
    learning_rate=lr,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.01,
    logging_strategy="epoch",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    fp16=True,
    optim="paged_adamw_8bit",

)

In [ ]:
# configure trainer
trainer = transformers.Trainer(
    model=model,
    train_dataset=train,
    eval_dataset=test,
    args=training_args,
    data_collator=data_collator)


# train model
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

# renable warnings
model.config.use_cache = True

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,2.016700,1.969901
2,2.094900,1.968105
3,2.070300,1.962376
4,2.067400,1.956320
5,2.005700,1.949980
6,2.053400,1.943136
7,1.991200,1.935859
8,2.033300,1.928465
9,1.993400,1.920955
10,2.035800,1.913550


config.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:429: UserWarning: torch.

### Проверяем результаты

In [ ]:
print(prompt)

[INST] Вы - русскоязычный ИИ ассистент, который помогает пользователям находить файлы, отвечать на их вопросы и поддерживать диалог. Вы имеете доступ к базе данных, которая автоматически покажет результаты поиска, если найдется какая-либо информация. Помните, что не все результаты поиска могут быть релевантны. Адаптируйтесь к запросам пользователей и предоставляйте понятные и полезные ответы на вопросы, поддерживая диалог на русском языке. 


Сообщение пользователя: 
Кто такой Иван Бездомный? 
[/INST]


In [ ]:
# tokenize input
inputs = tokenizer(prompt, return_tensors="pt")

# generate output
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=180)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Вы - русскоязычный ИИ ассистент, который помогает пользователям находить файлы, отвечать на их вопросы и поддерживать диалог. Вы имеете доступ к базе данных, которая автоматически покажет результаты поиска, если найдется какая-либо информация. Помните, что не все результаты поиска могут быть релевантны. Адаптируйтесь к запросам пользователей и предоставляйте понятные и полезные ответы на вопросы, поддерживая диалог на русском языке. 


Сообщение пользователя: 
Кто такой Иван Бездомный? 
[/INST] Иван Бездомный - это вымышленный персонаж из русской литературы. Он является одним из героев популярной русской сказки "Морозко" или "Снежанка" (в разных вариантах сказки его имя может звучать как Иван-да-Марьин, Иван-Царевич, Иван-Бездомный и т.д.). Этот персонаж известен своей доброй натурой, помогает главным героям сказки и спасает их от неприятностей. В сказке он часто одет в белые рубашки и живет в лесу. В некото


### Загрузка модели на Hugging Face

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

# # option 2: key login
# from huggingface_hub import login
# write_key = 'hf_' # paste token here
# login(write_key)

In [ ]:
hf_name = 'ggeorge' # your hf username or org name
model_id = hf_name + "/" + lora_model_name

In [ ]:
model.push_to_hub(model_id)
trainer.push_to_hub(model_id)

adapter_model.safetensors:   0%|          | 0.00/8.40M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/4.92k [00:00<?, ?B/s]

events.out.tfevents.1711874118.8413cff6e703.688.0:   0%|          | 0.00/34.0k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

CommitInfo(commit_url='https://huggingface.co/ggeorge/qlora-mistral-hackatone-yandexq/commit/e495d14229592d48b258241105e6d843cf71e24f', commit_message='ggeorge/qlora-mistral-hackatone-yandexq', commit_description='', oid='e495d14229592d48b258241105e6d843cf71e24f', pr_url=None, pr_revision=None, pr_num=None)

### Работа с fine-tuned моделью

In [ ]:
# load model from hub
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM

model_name = "TheBloke/Mistral-7B-Instruct-v0.2-GPTQ"
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="gptq-4bit-32g-actorder_True")

config = PeftConfig.from_pretrained("ggeorge/qlora-mistral-hackatone-yandexq")
model = PeftModel.from_pretrained(model, "ggeorge/qlora-mistral-hackatone-yandexq")

# load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=True)

In [ ]:
prompt = prompt_template('Что делать, если потерял паспорт? Как сделать новый?')
print(prompt)

[INST] Вы - русскоязычный ИИ ассистент, который помогает пользователям находить файлы, отвечать на их вопросы и поддерживать диалог. Вы имеете доступ к базе данных, которая автоматически покажет результаты поиска, если найдется какая-либо информация. Помните, что не все результаты поиска могут быть релевантны. Адаптируйтесь к запросам пользователей и предоставляйте понятные и полезные ответы на вопросы, поддерживая диалог на русском языке. 


Сообщение пользователя: 
Что делать, если потерял паспорт? Как сделать новый? 
[/INST]


In [ ]:
model.eval()

inputs = tokenizer(prompt, return_tensors="pt")
outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=280)

print(tokenizer.batch_decode(outputs)[0])

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


<s> [INST] Вы - русскоязычный ИИ ассистент, который помогает пользователям находить файлы, отвечать на их вопросы и поддерживать диалог. Вы имеете доступ к базе данных, которая автоматически покажет результаты поиска, если найдется какая-либо информация. Помните, что не все результаты поиска могут быть релевантны. Адаптируйтесь к запросам пользователей и предоставляйте понятные и полезные ответы на вопросы, поддерживая диалог на русском языке. 


Сообщение пользователя: 
Что делать, если потерял паспорт? Как сделать новый? 
[/INST] При потере паспорта необходимо сделать следующие шаги:

1. Сообщить о потере документа в органе внутренних дел. Для этого необходимо заполнить соответствующую форму и принести с собой документы, подтверждающие личность (например, водительские права, свидетельство о рождении, свидетельство о браке, свидетельство о регистрации жительства).
2. Заплатить плату за выдачу нового паспорта.
3. Получить временное удостоверение личности (в случае потери всех документо